In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import concurrent
import os,sys
from sklearn.metrics import precision_recall_curve,confusion_matrix,precision_score,recall_score,f1_score
from tqdm.notebook import tqdm

In [2]:
def get_files_scan_one_cpu(dir):
    """Get the path of all the files in nested folders"""
    all_files = []
    for entry in os.scandir(dir):
        if entry.is_file():
            all_files.append(entry.path)
        elif entry.is_dir():
            all_files.extend(get_files_scan(entry.path))
    if len(all_files) > 0:
        return all_files
    else:
        raise FileNotFoundError("No Files found. Please check your directory.")

def scan_files_in_folder(folder):
    """Scans files in a folder and returns a list of file paths."""
    files = []
    for entry in os.scandir(folder):
        if entry.is_file():
            files.append(entry.path)
        elif entry.is_dir():
            files.extend(scan_files_in_folder(entry.path))
    return files

def get_files_scan(dir,max_workers=1):
    """Get the path of all the files in nested folders using concurrent.futures."""
    all_files = []
    if max_workers==1:
        return get_files_scan_one_cpu(dir)
    else:
        with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
            future_to_dir = {executor.submit(scan_files_in_folder, entry.path): entry.path for entry in os.scandir(dir)}
            for future in concurrent.futures.as_completed(future_to_dir):
                folder = future_to_dir[future]
                try:
                    all_files.extend(future.result())
                except Exception as e:
                    print(f"An error occurred while scanning {folder}: {e}")

        if len(all_files) > 0:
            return all_files
        else:
            raise FileNotFoundError("No Files found. Please check your directory.")

In [4]:
CC3D_BDN_PATH="/mnt/isilon/mkhan/scan2brep/test/BRepEd/version_10/"
ABC_BDN_PATH_PIE="/mnt/isilon/mkhan/scan2brep_abl/test/BRepEd/version_0/"
ABC_BDN_PATH_PPL="/mnt/isilon/mkhan/scan2brep_abl/test/BRepEd/version_1/"

# CC3D - Boundary (Model Trained on CC3D and Tested on CC3D)

In [7]:
cc3d_bndry_gt_files=get_files_scan(os.path.join(CC3D_BDN_PATH,"bndry","gt"),4)
cc3d_bndry_pred_files=get_files_scan(os.path.join(CC3D_BDN_PATH,"bndry","pred"),4)

In [8]:
all_precision_scores=[]
all_recall_scores=[]
for i in tqdm(range(len(cc3d_bndry_gt_files))):
    gt_bndry=np.load(cc3d_bndry_gt_files[i])
    pred_bndry=np.load("/".join(cc3d_bndry_gt_files[i].split("/")[:-4]+["pred"]+\
                                cc3d_bndry_gt_files[i].split("/")[-3:]))
    pred_class=(pred_bndry>0.56).astype(int)
    all_precision_scores.append(precision_score(gt_bndry,pred_class))
    all_recall_scores.append(recall_score(gt_bndry,pred_class))

  0%|          | 0/4609 [00:00<?, ?it/s]

In [12]:
print(f"CC3D Boundary Detection Precision: {np.mean(all_precision_scores)}")
print(f"CC3D Boundary Detection Recall: {np.mean(all_recall_scores)}")

CC3D Boundary Detection Precision: 0.6517300129252261
CC3D Boundary Detection Recall: 0.6965228597627664


# CC3D - JNC (Model Trained on CC3D and Tested on CC3D)

In [13]:
cc3d_jnc_gt_files=get_files_scan(os.path.join("/mnt/isilon/mkhan/scan2brep/test/BRepJd/version_2/","jnc","gt"),4)
cc3d_jnc_pred_files=get_files_scan(os.path.join("/mnt/isilon/mkhan/scan2brep/test/BRepJd/version_2/","jnc","pred"),4)

In [14]:
all_precision_scores_jnc=[]
all_recall_scores_jnc=[]
all_f1_scores_jnc=[]
for i in tqdm(range(len(cc3d_jnc_gt_files))):
    gt_jnc=np.load(cc3d_jnc_gt_files[i])
    pred_jnc=np.load("/".join(cc3d_jnc_gt_files[i].split("/")[:-4]+["pred"]+\
                                cc3d_jnc_gt_files[i].split("/")[-3:]))
    pred_class=(pred_jnc>0.48).astype(int)
    all_precision_scores_jnc.append(precision_score(gt_jnc,pred_class))
    all_recall_scores_jnc.append(recall_score(gt_jnc,pred_class))
    all_f1_scores_jnc.append(f1_score(gt_jnc,pred_class))

  0%|          | 0/4609 [00:00<?, ?it/s]

/home/mkhan/.conda/envs/scan2brep/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mkhan/.conda/envs/scan2brep/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mkhan/.conda/envs/scan2brep/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mkhan/.conda/envs/scan2brep/lib/python3.8/site-packages

In [21]:
print(f"CC3D Junction Detection Precision: {np.mean(all_precision_scores_jnc)}")
print(f"CC3D Junction Detection Recall: {np.mean(all_recall_scores_jnc)}")

CC3D Junction Detection Precision: 0.42735551238813363
CC3D Junction Detection Recall: 0.36410591244602325


# ABC - BNDRY (Model Trained on ABC and Tested on ABC)

In [6]:
from torchmetrics.classification import PrecisionRecallCurve,BinaryPrecisionRecallCurve
import torch

In [5]:
abc_bndry_gt_files=get_files_scan(os.path.join(ABC_BDN_PATH_PIE,"bndry","gt"),4)
abc_bndry_pred_files=get_files_scan(os.path.join(ABC_BDN_PATH_PIE,"bndry","pred"),4)

In [23]:
# all_gt_bndry=[]
# all_pred_bndry=[]
# for i in tqdm(range(len(abc_bndry_gt_files))):
#     gt_bndry=np.load(abc_bndry_gt_files[i])
#     pred_bndry=np.load("/".join(abc_bndry_gt_files[i].split("/")[:-4]+["pred"]+\
#                                 abc_bndry_gt_files[i].split("/")[-3:]))
    
#     all_gt_bndry.append(list(gt_bndry))
#     all_pred_bndry.append(list(pred_bndry))

In [19]:
all_precision_scores_abc_bndry=[]
all_recall_scores_abc_bndry=[]
all_f1_scores_abc_bndry=[]
for i in tqdm(range(len(abc_bndry_gt_files))):
    gt_bndry=np.load(abc_bndry_gt_files[i])
    pred_bndry=np.load("/".join(abc_bndry_gt_files[i].split("/")[:-4]+["pred"]+\
                                abc_bndry_gt_files[i].split("/")[-3:]))
    pred_bndry=(pred_bndry>0.65).astype(int)
    all_precision_scores_abc_bndry.append(precision_score(gt_bndry,pred_bndry))
    all_recall_scores_abc_bndry.append(recall_score(gt_bndry,pred_bndry))
    all_f1_scores_abc_bndry.append(f1_score(gt_bndry,pred_bndry))

  0%|          | 0/955 [00:00<?, ?it/s]

In [25]:
# all_precision_scores_abc_bndry=[]
# all_recall_scores_abc_bndry=[]
# for i in range(len(abc_bndry_gt_files)):
#     gt_bndry=np.load(abc_bndry_gt_files[i])
#     pred_bndry=np.load("/".join(abc_bndry_gt_files[i].split("/")[:-4]+["pred"]+\
#                                 abc_bndry_gt_files[i].split("/")[-3:]))
    
#     all_precision_scores_abc_bndry.append(precision_score(gt_bndry,pred_bndry))
#     all_recall_scores_abc_bndry.append(recall_score(gt_bndry,pred_bndry))

In [21]:
print(f"ABC Boundary Detection Precision: {np.mean(all_precision_scores_abc_bndry)}")
print(f"ABC Boundary Detection Recall: {np.mean(all_recall_scores_abc_bndry)}")

ABC Boundary Detection Precision: 0.3383040532843028
ABC Boundary Detection Recall: 0.39126272962730513


In [28]:
abc_bndry_gt_files=get_files_scan(os.path.join(ABC_BDN_PATH_PPL,"bndry","gt"),4)
abc_bndry_pred_files=get_files_scan(os.path.join(ABC_BDN_PATH_PPL,"bndry","pred"),4)

all_precision_scores_abc_bndry=[]
all_recall_scores_abc_bndry=[]
all_f1_scores_abc_bndry=[]
for i in tqdm(range(len(abc_bndry_gt_files))):
    gt_bndry=np.load(abc_bndry_gt_files[i])
    pred_bndry=np.load("/".join(abc_bndry_gt_files[i].split("/")[:-4]+["pred"]+\
                                abc_bndry_gt_files[i].split("/")[-3:]))
    pred_bndry=(pred_bndry>0.65).astype(int)
    all_precision_scores_abc_bndry.append(precision_score(gt_bndry,pred_bndry))
    all_recall_scores_abc_bndry.append(recall_score(gt_bndry,pred_bndry))
    all_f1_scores_abc_bndry.append(f1_score(gt_bndry,pred_bndry))

  0%|          | 0/955 [00:00<?, ?it/s]

In [27]:
print(f"ABC Boundary Detection Precision: {np.mean(all_precision_scores_abc_bndry)}")
print(f"ABC Boundary Detection Recall: {np.mean(all_recall_scores_abc_bndry)}")

ABC Boundary Detection Precision: 0.4540812426786419
ABC Boundary Detection Recall: 0.23704346471014853


# Model Trained on ABC and Tested on CC3D

In [29]:
NEW_PATH="/mnt/isilon/mkhan/scan2brep/test/BRepEd/version_9/"
abc_bndry_gt_files=get_files_scan(os.path.join(NEW_PATH,"bndry","gt"),4)
abc_bndry_pred_files=get_files_scan(os.path.join(NEW_PATH,"bndry","pred"),4)

all_precision_scores_abc_bndry=[]
all_recall_scores_abc_bndry=[]
all_f1_scores_abc_bndry=[]
for i in tqdm(range(len(abc_bndry_gt_files))):
    gt_bndry=np.load(abc_bndry_gt_files[i])
    pred_bndry=np.load("/".join(abc_bndry_gt_files[i].split("/")[:-4]+["pred"]+\
                                abc_bndry_gt_files[i].split("/")[-3:]))
    pred_bndry=(pred_bndry>0.56).astype(int)
    all_precision_scores_abc_bndry.append(precision_score(gt_bndry,pred_bndry))
    all_recall_scores_abc_bndry.append(recall_score(gt_bndry,pred_bndry))
    all_f1_scores_abc_bndry.append(f1_score(gt_bndry,pred_bndry))

  0%|          | 0/4609 [00:00<?, ?it/s]

In [33]:
print(f"ABC Boundary Detection Recall: {np.mean(all_recall_scores_abc_bndry).round(3)}")
print(f"ABC Boundary Detection Precision: {np.mean(all_precision_scores_abc_bndry).round(3)}")

ABC Boundary Detection Recall: 0.747
ABC Boundary Detection Precision: 0.631


In [39]:
NEW_PATH="/mnt/isilon/mkhan/scan2brep_abl/test/BRepEd/version_2"
abc_bndry_gt_files=get_files_scan(os.path.join(NEW_PATH,"bndry","gt"),4)
abc_bndry_pred_files=get_files_scan(os.path.join(NEW_PATH,"bndry","pred"),4)

all_precision_scores_abc_bndry=[]
all_recall_scores_abc_bndry=[]
all_f1_scores_abc_bndry=[]
for i in tqdm(range(len(abc_bndry_gt_files))):
    gt_bndry=np.load(abc_bndry_gt_files[i])
    pred_bndry=np.load("/".join(abc_bndry_gt_files[i].split("/")[:-4]+["pred"]+\
                                abc_bndry_gt_files[i].split("/")[-3:]))
    pred_bndry=(pred_bndry>0.65).astype(int)
    all_precision_scores_abc_bndry.append(precision_score(gt_bndry,pred_bndry))
    all_recall_scores_abc_bndry.append(recall_score(gt_bndry,pred_bndry))
    all_f1_scores_abc_bndry.append(f1_score(gt_bndry,pred_bndry))

  0%|          | 0/4609 [00:00<?, ?it/s]

/home/mkhan/.conda/envs/scan2brep/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mkhan/.conda/envs/scan2brep/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mkhan/.conda/envs/scan2brep/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mkhan/.conda/envs/scan2brep/lib/python3.8/site-packages

In [41]:
print(f"ABC Boundary Detection Recall: {np.mean(all_recall_scores_abc_bndry).round(3)}")
print(f"ABC Boundary Detection Precision: {np.mean(all_precision_scores_abc_bndry).round(3)}")

ABC Boundary Detection Recall: 0.43
ABC Boundary Detection Precision: 0.529


In [43]:
NEW_PATH="/mnt/isilon/mkhan/scan2brep_abl/test/BRepEd/version_3"
abc_bndry_gt_files=get_files_scan(os.path.join(NEW_PATH,"bndry","gt"),4)
abc_bndry_pred_files=get_files_scan(os.path.join(NEW_PATH,"bndry","pred"),4)

all_precision_scores_abc_bndry=[]
all_recall_scores_abc_bndry=[]
all_f1_scores_abc_bndry=[]
for i in tqdm(range(len(abc_bndry_gt_files))):
    gt_bndry=np.load(abc_bndry_gt_files[i])
    pred_bndry=np.load("/".join(abc_bndry_gt_files[i].split("/")[:-4]+["pred"]+\
                                abc_bndry_gt_files[i].split("/")[-3:]))
    pred_bndry=(pred_bndry>0.65).astype(int)
    all_precision_scores_abc_bndry.append(precision_score(gt_bndry,pred_bndry))
    all_recall_scores_abc_bndry.append(recall_score(gt_bndry,pred_bndry))
    all_f1_scores_abc_bndry.append(f1_score(gt_bndry,pred_bndry))

  0%|          | 0/4609 [00:00<?, ?it/s]

/home/mkhan/.conda/envs/scan2brep/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mkhan/.conda/envs/scan2brep/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mkhan/.conda/envs/scan2brep/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mkhan/.conda/envs/scan2brep/lib/python3.8/site-packages

In [44]:
print(f"ABC Boundary Detection Recall: {np.mean(all_recall_scores_abc_bndry).round(3)}")
print(f"ABC Boundary Detection Precision: {np.mean(all_precision_scores_abc_bndry).round(3)}")

ABC Boundary Detection Recall: 0.383
ABC Boundary Detection Precision: 0.622


# Model Trained on CC3D and Tested on ABC

In [34]:
NEW_PATH="/mnt/isilon/mkhan/scan2brep/test/BRepEd/version_11/"
abc_bndry_gt_files=get_files_scan(os.path.join(NEW_PATH,"bndry","gt"),4)
abc_bndry_pred_files=get_files_scan(os.path.join(NEW_PATH,"bndry","pred"),4)

all_precision_scores_abc_bndry=[]
all_recall_scores_abc_bndry=[]
all_f1_scores_abc_bndry=[]
for i in tqdm(range(len(abc_bndry_gt_files))):
    gt_bndry=np.load(abc_bndry_gt_files[i])
    pred_bndry=np.load("/".join(abc_bndry_gt_files[i].split("/")[:-4]+["pred"]+\
                                abc_bndry_gt_files[i].split("/")[-3:]))
    pred_bndry=(pred_bndry>0.56).astype(int)
    all_precision_scores_abc_bndry.append(precision_score(gt_bndry,pred_bndry))
    all_recall_scores_abc_bndry.append(recall_score(gt_bndry,pred_bndry))
    all_f1_scores_abc_bndry.append(f1_score(gt_bndry,pred_bndry))

  0%|          | 0/955 [00:00<?, ?it/s]

In [36]:
print(f"ABC Boundary Detection Recall: {np.mean(all_recall_scores_abc_bndry).round(4)}")
print(f"ABC Boundary Detection Precision: {np.mean(all_precision_scores_abc_bndry).round(3)}")

ABC Boundary Detection Recall: 0.3701
ABC Boundary Detection Precision: 0.51


In [60]:
NEW_PATH="/mnt/isilon/mkhan/scan2brep_abl/test/BRepEd/version_6/"
abc_bndry_gt_files=get_files_scan(os.path.join(NEW_PATH,"bndry","gt"),4)
abc_bndry_pred_files=get_files_scan(os.path.join(NEW_PATH,"bndry","pred"),4)

all_precision_scores_abc_bndry=[]
all_recall_scores_abc_bndry=[]
all_f1_scores_abc_bndry=[]
for i in tqdm(range(len(abc_bndry_gt_files))):
    gt_bndry=np.load(abc_bndry_gt_files[i])
    pred_bndry=np.load("/".join(abc_bndry_gt_files[i].split("/")[:-4]+["pred"]+\
                                abc_bndry_gt_files[i].split("/")[-3:]))
    pred_bndry=(pred_bndry>0.57).astype(int)
    all_precision_scores_abc_bndry.append(precision_score(gt_bndry,pred_bndry))
    all_recall_scores_abc_bndry.append(recall_score(gt_bndry,pred_bndry))
    all_f1_scores_abc_bndry.append(f1_score(gt_bndry,pred_bndry))

  0%|          | 0/955 [00:00<?, ?it/s]

/home/mkhan/.conda/envs/scan2brep/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/mkhan/.conda/envs/scan2brep/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [59]:
print(f"ABC Boundary Detection Recall: {np.mean(all_recall_scores_abc_bndry).round(4)}")
print(f"ABC Boundary Detection Precision: {np.mean(all_precision_scores_abc_bndry).round(3)}")

ABC Boundary Detection Recall: 0.4407
ABC Boundary Detection Precision: 0.459


In [63]:
NEW_PATH="/mnt/isilon/mkhan/scan2brep_abl/test/BRepEd/version_7/"
abc_bndry_gt_files=get_files_scan(os.path.join(NEW_PATH,"bndry","gt"),4)
abc_bndry_pred_files=get_files_scan(os.path.join(NEW_PATH,"bndry","pred"),4)

all_precision_scores_abc_bndry=[]
all_recall_scores_abc_bndry=[]
all_f1_scores_abc_bndry=[]
for i in tqdm(range(len(abc_bndry_gt_files))):
    gt_bndry=np.load(abc_bndry_gt_files[i])
    pred_bndry=np.load("/".join(abc_bndry_gt_files[i].split("/")[:-4]+["pred"]+\
                                abc_bndry_gt_files[i].split("/")[-3:]))
    pred_bndry=(pred_bndry>0.56).astype(int)
    all_precision_scores_abc_bndry.append(precision_score(gt_bndry,pred_bndry))
    all_recall_scores_abc_bndry.append(recall_score(gt_bndry,pred_bndry))
    all_f1_scores_abc_bndry.append(f1_score(gt_bndry,pred_bndry))

  0%|          | 0/955 [00:00<?, ?it/s]

In [65]:
print(f"ABC Boundary Detection Recall: {np.mean(all_recall_scores_abc_bndry).round(4)}")
print(f"ABC Boundary Detection Precision: {np.mean(all_precision_scores_abc_bndry).round(3)}")

ABC Boundary Detection Recall: 0.3801
ABC Boundary Detection Precision: 0.502


# Model Trained on CC3D and Tested on CC3D

In [46]:
NEW_PATH="/mnt/isilon/mkhan/scan2brep_abl/test/BRepEd/version_4/"
abc_bndry_gt_files=get_files_scan(os.path.join(NEW_PATH,"bndry","gt"),4)
abc_bndry_pred_files=get_files_scan(os.path.join(NEW_PATH,"bndry","pred"),4)

all_precision_scores_abc_bndry=[]
all_recall_scores_abc_bndry=[]
all_f1_scores_abc_bndry=[]
for i in tqdm(range(len(abc_bndry_gt_files))):
    gt_bndry=np.load(abc_bndry_gt_files[i])
    pred_bndry=np.load("/".join(abc_bndry_gt_files[i].split("/")[:-4]+["pred"]+\
                                abc_bndry_gt_files[i].split("/")[-3:]))
    pred_bndry=(pred_bndry>0.56).astype(int)
    all_precision_scores_abc_bndry.append(precision_score(gt_bndry,pred_bndry))
    all_recall_scores_abc_bndry.append(recall_score(gt_bndry,pred_bndry))
    all_f1_scores_abc_bndry.append(f1_score(gt_bndry,pred_bndry))

  0%|          | 0/4609 [00:00<?, ?it/s]

In [48]:
print(f"ABC Boundary Detection Recall: {np.mean(all_recall_scores_abc_bndry).round(4)}")
print(f"ABC Boundary Detection Precision: {np.mean(all_precision_scores_abc_bndry).round(3)}")

ABC Boundary Detection Recall: 0.6767
ABC Boundary Detection Precision: 0.629


In [54]:
NEW_PATH="/mnt/isilon/mkhan/scan2brep_abl/test/BRepEd/version_5/"
abc_bndry_gt_files=get_files_scan(os.path.join(NEW_PATH,"bndry","gt"),4)
abc_bndry_pred_files=get_files_scan(os.path.join(NEW_PATH,"bndry","pred"),4)

all_precision_scores_abc_bndry=[]
all_recall_scores_abc_bndry=[]
all_f1_scores_abc_bndry=[]
for i in tqdm(range(len(abc_bndry_gt_files))):
    gt_bndry=np.load(abc_bndry_gt_files[i])
    pred_bndry=np.load("/".join(abc_bndry_gt_files[i].split("/")[:-4]+["pred"]+\
                                abc_bndry_gt_files[i].split("/")[-3:]))
    pred_bndry=(pred_bndry>0.56).astype(int)
    all_precision_scores_abc_bndry.append(precision_score(gt_bndry,pred_bndry))
    all_recall_scores_abc_bndry.append(recall_score(gt_bndry,pred_bndry))
    all_f1_scores_abc_bndry.append(f1_score(gt_bndry,pred_bndry))

  0%|          | 0/4609 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [53]:
print(f"ABC Boundary Detection Recall: {np.mean(all_recall_scores_abc_bndry).round(4)}")
print(f"ABC Boundary Detection Precision: {np.mean(all_precision_scores_abc_bndry).round(3)}")

ABC Boundary Detection Recall: 0.7291
ABC Boundary Detection Precision: 0.636


# Pienet Junction

In [69]:
import warnings
warnings.filterwarnings("ignore")

In [92]:
NEW_PATH="/mnt/isilon/mkhan/scan2brep_pienet/test/BRepJd/version_7/"
abc_jnc_gt_files=get_files_scan(os.path.join(NEW_PATH,"jnc","gt"),4)
abc_jnc_pred_files=get_files_scan(os.path.join(NEW_PATH,"jnc","pred"),4)

all_precision_scores_abc_jnc=[]
all_recall_scores_abc_jnc=[]
all_f1_scores_abc_jnc=[]
for i in tqdm(range(len(abc_jnc_gt_files))):
    gt_jnc=np.load(abc_jnc_gt_files[i])
    pred_jnc=np.load("/".join(abc_jnc_gt_files[i].split("/")[:-4]+["pred"]+\
                                abc_jnc_gt_files[i].split("/")[-3:]))
    pred_jnc=(pred_jnc>0.5).astype(int)
    all_precision_scores_abc_jnc.append(precision_score(gt_jnc,pred_jnc))
    all_recall_scores_abc_jnc.append(recall_score(gt_jnc,pred_jnc))
    all_f1_scores_abc_jnc.append(f1_score(gt_jnc,pred_jnc))

  0%|          | 0/954 [00:00<?, ?it/s]

In [94]:
print(f"ABC Boundary Detection Recall: {np.mean(all_precision_scores_abc_jnc).round(4)}")
print(f"ABC Boundary Detection Precision: {np.mean(all_f1_scores_abc_jnc).round(3)}")

ABC Boundary Detection Recall: 0.0
ABC Boundary Detection Precision: 0.0


In [97]:
val=torch.load("/home/mkhan/Codes/Scan-to-BRep/Scan2BRep_PreTrainedModels/ABC/BRepJd/train/sapphire/version_6/checkpoint/last.ckpt")

In [98]:
val

{'epoch': 0,
 'global_step': 606,
 'pytorch-lightning_version': '1.7.7',
 'state_dict': OrderedDict([('emb_nn.bn1.weight',
               tensor([  1140090.6250,   7100925.5000, -16350145.0000, -24534892.0000,
                        -9458505.0000,  32198216.0000,   7442870.5000,  64667428.0000,
                        -5853588.5000,  -1388997.3750,  11750791.0000,   8202881.0000,
                       -26388092.0000, -15086011.0000,  -3942235.7500, -16239972.0000,
                        14324107.0000,    -90975.0234,  -2073226.6250,   1446804.0000,
                        21947910.0000,   -454278.6250,  13397209.0000,   6377932.5000,
                        28755996.0000, -22027688.0000, -24861140.0000,   4788726.5000,
                        17108730.0000,  -6765361.5000,  -9206152.0000,   5805780.5000,
                        -1074118.7500, -10408275.0000,  11272911.0000,   -245521.0625,
                        -6385233.0000,  17215586.0000,  -2268552.5000,   -836843.8750,
       